In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./project-public.csv')

In [3]:
df.head()

,EEID,Status,Age,Gender,Education Level,Hire Date,Termination Date,Termination Type,Termination Reason,Job Information: Date,...,Department,Supervisor EID,Pay rate,Pay rate - Currency code,Compensation Change Reason,Stock Options,Level,Is Supervisor,Length of service,Length of service: Years
0,343,Inactive,28,Female,Undergraduate degree (Grado),19 Aug 2019,30 Sep 2019,Involuntary,End of contract,19 Aug 2019,...,People Operations,1.0,6047,EUR,Reduce/Increase Working hours,N,NaN,No,"1 month, 11 days",0
1,468,Inactive,29,Male,Unknown,05 Jul 2016,30 Mar 2017,Involuntary,End of contract,05 Jul 2016,...,NaN,234.0,6144,EUR,NaN,N,NaN,No,"8 months, 25 days",0
2,458,Inactive,27,Male,Unknown,05 Sep 2016,26 Jan 2017,Involuntary,End of contract,05 Sep 2016,...,NaN,NaN,6144,EUR,NaN,N,NaN,No,"4 months, 21 days",0
3,379,Inactive,25,Male,Undergraduate degree (Grado),23 Oct 2017,28 Aug 2018,Involuntary,End of contract,23 Oct 2017,...,NaN,NaN,6144,EUR,New hire,N,NaN,No,"10 months, 5 days",0
4,492,Inactive,58,Female,Unknown,13 Sep 2017,29 Sep 2017,Involuntary,End of contract,13 Sep 2017,...,NaN,NaN,7200,EUR,NaN,N,NaN,No,16 days,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   EEID                        607 non-null    int64  
 1   Status                      607 non-null    object 
 2   Age                         607 non-null    int64  
 3   Gender                      606 non-null    object 
 4   Education Level             602 non-null    object 
 5   Hire Date                   607 non-null    object 
 6   Termination Date            258 non-null    object 
 7   Termination Type            258 non-null    object 
 8   Termination Reason          254 non-null    object 
 9   Job Information: Date       607 non-null    object 
 10  Division                    604 non-null    object 
 11  Department                  520 non-null    object 
 12  Supervisor EID              500 non-null    float64
 13  Pay rate                    607 non

In [5]:
df.set_index(df['EEID'], inplace=True)
df.drop(columns='EEID', inplace=True)

In [6]:
df['Hire Date'] = pd.to_datetime(df['Hire Date'])

In [7]:
df['Termination Date'] = pd.to_datetime(df['Termination Date'])

In [8]:
df['Job Information: Date'] = pd.to_datetime(df['Job Information: Date'])

In [9]:
df['Education Level'].fillna('Unknown', inplace=True)

In [10]:
df['Gender'].fillna('Unknown', inplace=True)

In [11]:
df['Division'].fillna('Unknown', inplace=True)

In [12]:
df['Department'].fillna('Unknown', inplace=True)

In [13]:
df['Compensation Change Reason'].fillna('Unknown', inplace=True)

In [14]:
df['Supervisor EID'].fillna(0, inplace=True)

In [15]:
df['Level'].fillna('Unknown', inplace=True)

In [29]:
df = df.astype({'Supervisor EID': 'int', 'Pay rate': 'float64'})

In [27]:
df.head()

,Status,Age,Gender,Education Level,Hire Date,Termination Date,Termination Type,Termination Reason,Job Information: Date,Division,...,Supervisor EID,Pay rate,Pay rate - Currency code,Compensation Change Reason,Stock Options,Level,Is Supervisor,Length of service,Length of service: Years,Length of service: Months
EEID,,,,,,,,,,,,,,,,,,,,,
343,Inactive,28,Female,Undergraduate degree (Grado),2019-08-19,2019-09-30,Involuntary,End of contract,2019-08-19,People,...,1.0,6047,EUR,Reduce/Increase Working hours,N,Unknown,No,"1 month, 11 days",0,1
468,Inactive,29,Male,Unknown,2016-07-05,2017-03-30,Involuntary,End of contract,2016-07-05,Engineering,...,234.0,6144,EUR,Unknown,N,Unknown,No,"8 months, 25 days",0,8
458,Inactive,27,Male,Unknown,2016-09-05,2017-01-26,Involuntary,End of contract,2016-09-05,Engineering,...,0.0,6144,EUR,Unknown,N,Unknown,No,"4 months, 21 days",0,4
379,Inactive,25,Male,Undergraduate degree (Grado),2017-10-23,2018-08-28,Involuntary,End of contract,2017-10-23,Engineering,...,0.0,6144,EUR,New hire,N,Unknown,No,"10 months, 5 days",0,10
492,Inactive,58,Female,Unknown,2017-09-13,2017-09-29,Involuntary,End of contract,2017-09-13,People,...,0.0,7200,EUR,Unknown,N,Unknown,No,16 days,0,0


In [26]:
today = pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))
length = (df['Termination Date'].fillna(today) - df['Hire Date']) / 30
df['Length of service: Months'] = length.astype(str).apply(lambda x: int(x.split(' ')[0]))

# for index, row in df.iterrows():
#     if row['Termination Date'] is not pd.NaT:
#         months = (row['Termination Date'] - row['Hire Date']) / 30
#         
#         row['Length of service: Months'].append(data)
#     else:
#         months = (today - row['Hire Date']) / 30
#         data = int(str(months).split(' ')[0])
#         row['Length of service: Months'].append(data) 

Pasar variables categóricas a numéricas con el one-hot-encoding o con el get dummies

# Scaler

In [ ]:
df_scale = df[['Pay rate']]
df_scale.head()

In [ ]:
scale = StandardScaler()

In [ ]:
scale.fit_transform(df_scale)